# Simulation

In [32]:
import numpy as np
import scipy.stats
from math import log
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"
from scipy import stats

### Generating a random sample from a discrete distribution

numpy.random.choice() function:

Put the outcomes in the first argument, a.
Put the desired sample size in the argument size.
Put the probabilities of the outcomes respective to x in the argument p.

In [33]:
np.random.seed(1)
outcomes = ["banana", "bob-omb", "coin", "horn", "shell"]
probs = [0.12, 0.05, 0.75, 0.03, 0.05]
n = 10
np.random.choice(outcomes, size = n, p = probs)

array(['coin', 'coin', 'banana', 'coin', 'bob-omb', 'banana', 'coin',
       'coin', 'coin', 'coin'], dtype='<U7')

### Generating Samples from Distribution Families

1. Binomial - scipy.stats.binom.rvs()
2. Geometric - scipy.stats.geom.rvs()
3. Negative Binomial - scipy.stats.nbinom.rvs()
4. Poisson - scipy.stats.poisson.rvs() 



In [34]:
import scipy.stats

scipy.stats.binom.rvs(n=5, p=0.6, size=10)

array([3, 2, 4, 2, 5, 3, 3, 3, 4, 4])

Consider playing games with probability of success $p=0.7$ until you experience $k=5$ successes, and counting the number of failures. This random variable (say $X$) has a Negative Binomial distribution.

Let's demonstrate both a distribution-based and empirical approach to computing the variance and pmf. First, let's obtain our random sample (of, say, 10000 observations).

In [35]:
sim_sample = scipy.stats.nbinom.rvs(n=5, p=0.7, size=10000)
sim_sample

array([2, 1, 4, ..., 1, 2, 2])

In [36]:
# True distribution based
p = 0.7
k = 5
n = 10000

mean = (1-p) * k/p
variance = (1 - p) * k / p**2
mean, variance

(2.1428571428571432, 3.061224489795919)

In [37]:
# Approximate empirical based
sim_sample.mean(), sim_sample.var()

(2.1448, 3.01523296)

In [38]:
mean - sim_sample.mean(), variance - sim_sample.var() 

(-0.001942857142856802, 0.04599152979591903)

In [39]:
def entropy2(labels, base=None):
  """ Computes entropy of label distribution. """

  n_labels = len(labels)

  if n_labels <= 1:
    return 0

  value, counts = np.unique(labels, return_counts=True)
  probs = counts / n_labels
  n_classes = np.count_nonzero(probs)

  if n_classes <= 1:
    return 0

  ent = 0.

  for i in probs:
    ent -= i * log(i)

  return ent

In [40]:
entropy2(sim_sample)

1.8543618550256897

### How Sample Size allows us to get closer to the Actual Mean.

In [41]:
sample_sizes = [1*(2**x) for x in range(12)]
sample_sizes

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]

In [42]:
np.random.seed(1)
outcomes = [1, 2, 3, 4, 5]
probs = [0.12, 0.3, 0.25, 0.28, 0.05]

means = []
variances = []
for sample_size in sample_sizes:
    sample = np.random.choice(outcomes, size=sample_size, p=probs)
    means.append(sample.mean())
    variances.append(sample.var())

In [43]:
fig = px.line(x=sample_sizes, y=means)
fig.show()

The idea is that some random variables will have a distribution that depends on other random variables, but in a way that's explicit. For example, consider a random variable $T$ that we can obtain as follows. Take $X \sim \text{Poisson}(5)$, and then $T = \sum_{i = 1}^{X} D_i$, where each $D_i$ are iid with some specified distribution. In this case, to generate $T$, you would first need to generate $X$, then generate $X$ values of $D_i$, then sum those up to get $T$. This is the example we'll see here, but in general, you can have any number of dependencies, each component of which you would have to generate.

Consider an example that a Vancouver port faces with "gang demand". Whenever a ship arrives to the port of Vancouver, they request a certain number of "gangs" (groups of people) to help unload the ship. Let's suppose the number of gangs requested by a ship has the following distribution:

In [44]:
gang = range(1, 5)
gang_probs = [0.2, 0.4, 0.3, 0.1]
px.bar(x=gang, y=gang_probs, title='Numbers of Gangs and their Probabilitity')

In [45]:
# Testing that np.choice is working as expected

def gang_demand(n_ships, gangs=gang, probs=gang_probs):
    sample = np.random.choice(gangs, size=n_ships, p=probs)
    return sample.sum()

gang_demand(10)

23

Let's say that the number of ships arriving each day follows a poisson distribution with lambda=5.
1. Let's simulate 10k random observations of ship_arrivals 
2. Calculate how many gangs were demanded for each day

In [50]:
lambda_ = 5
samples = 10000
arrivals = stats.poisson.rvs(lambda_, size=samples)
gang_demand = []
for arrival in arrivals:
    day_sum = 0
    for i in range(arrival):
        day_sum += np.random.choice(gang, p=gang_probs)
    gang_demand.append(day_sum)


1. Calculate the pmf for gang demand

In [51]:
gang_demand_count = {}
for gangs in gang_demand:
    if gangs not in gang_demand_count:
        gang_demand_count[gangs] = 1
    if gangs in gang_demand_count:
        gang_demand_count[gangs] += 1

gang_demand_count

{17: 367,
 5: 451,
 6: 546,
 18: 323,
 20: 189,
 15: 499,
 7: 641,
 13: 615,
 9: 747,
 11: 699,
 10: 763,
 0: 66,
 19: 237,
 21: 145,
 8: 695,
 4: 362,
 14: 577,
 12: 685,
 2: 161,
 3: 249,
 16: 487,
 25: 53,
 23: 68,
 22: 135,
 26: 48,
 24: 65,
 27: 28,
 30: 9,
 1: 67,
 38: 3,
 31: 10,
 28: 21,
 29: 10,
 37: 4,
 41: 2,
 33: 4,
 34: 3,
 32: 4}

In [52]:
for gang in gang_demand_count:
    gang_demand_count[gang] = gang_demand_count[gang] / samples
gang_demand_count

{17: 0.0367,
 5: 0.0451,
 6: 0.0546,
 18: 0.0323,
 20: 0.0189,
 15: 0.0499,
 7: 0.0641,
 13: 0.0615,
 9: 0.0747,
 11: 0.0699,
 10: 0.0763,
 0: 0.0066,
 19: 0.0237,
 21: 0.0145,
 8: 0.0695,
 4: 0.0362,
 14: 0.0577,
 12: 0.0685,
 2: 0.0161,
 3: 0.0249,
 16: 0.0487,
 25: 0.0053,
 23: 0.0068,
 22: 0.0135,
 26: 0.0048,
 24: 0.0065,
 27: 0.0028,
 30: 0.0009,
 1: 0.0067,
 38: 0.0003,
 31: 0.001,
 28: 0.0021,
 29: 0.001,
 37: 0.0004,
 41: 0.0002,
 33: 0.0004,
 34: 0.0003,
 32: 0.0004}

In [54]:
px.bar(x=gang_demand_count.keys(), y=gang_demand_count.values(), title='Probability of # of Gangs Demanded')